# Urbanpy model for Para


Start date: 2022-02-09
Make sure the necessary packages are installed from the `pipenv` file created for this work.

To do this, go to the local directory for this project, and in your terminal run:

`pipenv install`

Then, activate the environment by running

`pipenv shell`

**Note**: You may need to run `brew install gdal` in terminal and then `pip install urbanpy` in this notebook to get code below to run. Issue submitted about installation difficulties [here](https://github.com/EL-BID/urbanpy/issues/18).


In [1]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 10)

# Only needed when git cloning the urbanpy repo
# import sys
# sys.path.append('..')

import urbanpy as up
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import os
import time

from tqdm.notebook import tqdm

tqdm.pandas()

from pandarallel import pandarallel

cores = os.cpu_count()

# Adjust the number of workers to your setup (computer & docker)
pandarallel.initialize(progress_bar=True, nb_workers=cores)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import h3
from shapely.geometry import Polygon

## Read Para's hexagons


In [3]:
para_hexs = pd.read_csv("outputs/29112023_para_hexs_final.csv", index_col=0)

In [4]:
para_hexs.head()

,hex,population_2020,pop_3_5_years_adj,pop_6_14_years_adj,pop_15_17_years_adj,pop_18_years_adj,V002_adj,V003_adj,income_pc,ensino_fundamental,...,AVG_QT_TUR_MED,AVG_QT_TUR_PROF,AVG_QT_TUR_PROF_TEC,AVG_QT_TUR_EJA,AVG_QT_TUR_EJA_FUND,AVG_QT_TUR_EJA_MED,AVG_QT_TUR_ESP,AVG_QT_TUR_ESP_CC,AVG_QT_TUR_ESP_CE,urban_area
0,888062d73dfffff,328.677081,464.000000,74.000000,7.000000,2.000000,1741.000000,110501.000000,63.469845,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0
25,8880669749fffff,6.645306,0.620557,0.214808,0.031823,0.039779,6.491976,1138.092715,175.307598,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
35,888a92cb25fffff,19.246575,10.420928,4.408854,1.002012,0.200402,61.122752,14736.595316,241.098361,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
40,8881439147fffff,5.421374,0.617580,0.420769,0.067866,0.020360,4.044809,801.679859,198.199664,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
46,88806dc2b3fffff,6.842970,4.088244,1.233839,0.257817,0.073662,25.229252,2125.739460,84.256934,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
para_hexs.shape

(132361, 193)

In [6]:
para_hexs.columns

Index(['hex', 'population_2020', 'pop_3_5_years_adj', 'pop_6_14_years_adj',
       'pop_15_17_years_adj', 'pop_18_years_adj', 'V002_adj', 'V003_adj',
       'income_pc', 'ensino_fundamental',
       ...
       'AVG_QT_TUR_MED', 'AVG_QT_TUR_PROF', 'AVG_QT_TUR_PROF_TEC',
       'AVG_QT_TUR_EJA', 'AVG_QT_TUR_EJA_FUND', 'AVG_QT_TUR_EJA_MED',
       'AVG_QT_TUR_ESP', 'AVG_QT_TUR_ESP_CC', 'AVG_QT_TUR_ESP_CE',
       'urban_area'],
      dtype='object', length=193)

In [7]:
%%time
# Get polygon from h3 index
para_hexs["geometry"] = para_hexs["hex"].parallel_apply(
    lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True))
)

CPU times: user 393 ms, sys: 138 ms, total: 531 ms
Wall time: 883 ms


In [8]:
para_hexs = gpd.GeoDataFrame(para_hexs, crs="4326")

## Visualize the population data


In [9]:
if False:
    fig = up.plotting.choropleth_map(
        para_hexs.reset_index(drop=True),
        "population_2020",
        title="Para Population - 2020",
        zoom=8,
        color_continuous_scale="Viridis",
        opacity=0.1,
        labels={"population_2020": "Pop. 2020"},
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
    )
    fig.update_traces(marker_line_width=0)
    fig.show()

## Urban accessibility - import high quality school data


These data are stored in a csv file named `brazil_schools_census_education_metrics.parquet` that we need to read.


In [10]:
br_schools = gpd.read_parquet("outputs/brazil_schools_census_edu_metrics.parquet")

In [11]:
br_schools.shape

(222936, 147)

In [12]:
br_schools.head(2)

,abbrev_state,name_muni,code_school,name_school,education_level,education_level_others,admin_category,address,phone_number,government_level,...,FUN_07_CAT_0,FUN_08_CAT_0,FUN_09_CAT_0,MULT_ETA_CAT_0,MED_CAT_0,MED_01_CAT_0,MED_02_CAT_0,MED_03_CAT_0,MED_04_CAT_0,MED_NS_CAT_0
0,RO,Porto Velho,11000023,EEEE ABNAEL MACHADO DE LIMA - CENE,Ensino Fundamental,Atendimento Educacional Especializado,Pública,"AVENIDA AMAZONAS, 6492 ZONA LESTE. TIRADENTES....",(69) 992083054,Estadual,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RO,Porto Velho,11000040,EMEIEF PEQUENOS TALENTOS,Educação Infantil,,Pública,"RUA CAETANO, 3256 PREDIO. CALADINHO. 76808-108...",(69) 32135237,Municipal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Apply the filters
# Only schools that are active
active_filter = (
    br_schools["service_restriction"]
    == "ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO"
)
state_filter = br_schools["abbrev_state"] == "PA"
filtered_schools = br_schools[active_filter & state_filter]

In [14]:
# Print a small report with the number of schools and percentage of the total
print(f"Total number of schools: {len(br_schools)}")
print(f"Number of schools selected: {len(filtered_schools)}")
print(f"Percentage of the total: {len(filtered_schools) / len(br_schools) * 100:.2f}%")

Total number of schools: 222936
Number of schools selected: 10755
Percentage of the total: 4.82%


## Evaluate Accessibility


In [15]:
filtered_schools["lat"] = filtered_schools.geometry.y
filtered_schools["lon"] = filtered_schools.geometry.x

In [16]:
filtered_schools.geometry.x.isna().sum()

2404

In [17]:
filtered_schools = filtered_schools.dropna(subset=["lat", "lon"])

In [18]:
para_hexs["lat"] = para_hexs.geometry.centroid.y
para_hexs["lon"] = para_hexs.geometry.centroid.x

Get the nearest school from each hexagons centroid


In [19]:
dist_up, ind_up = up.utils.nn_search(
    # These are the schools
    tree_features=filtered_schools[["lat", "lon"]].values,
    # Values are the centroids of each hexagon
    query_features=para_hexs[["lat", "lon"]].values,
    metric="haversine",
)

This adds new column to indicate the index of the closest school for a particular hexagon


In [20]:
para_hexs["closest_school_id"] = ind_up
para_hexs["closest_school_dist"] = dist_up

## Download data needed for Para


In [21]:
!cd ~/data/osrm && wget -nc https://download.geofabrik.de/south-america/brazil/norte-latest.osm.pbf

File ‘norte-latest.osm.pbf’ already there; not retrieving.



## Start the OSRM server


In [22]:
# Download unix_download.sh file from github repo
!cd .env/lib/python3.11/site-packages/urbanpy/routing/ && wget -nc https://raw.githubusercontent.com/EL-BID/urbanpy/master/urbanpy/routing/unix_download.sh

File ‘unix_download.sh’ already there; not retrieving.



In [23]:
# Set the travel profile
PROFILE = "foot"

In [24]:
up.routing.start_osrm_server("norte", "south-america_brazil", PROFILE)

Starting server ...
osrm_routing_server_south-america_brazil_norte_foot
Server was started succesfully


Then we can do our distance and duration calculations


In [25]:
# Running an example route to check if the server is working
# This also fixs the Python multiprocessing error in MacOS
# (For more info see: https://github.com/nalepae/pandarallel/issues/225)

origin = para_hexs.iloc[0]
destination = filtered_schools.iloc[para_hexs.iloc[0]["closest_school_id"]]

# (distance in meters, duration in seconds)
distance, duration = up.routing.osrm_route(
    origin.geometry.centroid, destination.geometry
)

print(
    f"""
OSRM results using profile={PROFILE}
------------
Origin: {origin.geometry.centroid.y, origin.geometry.centroid.x},
Destination: {destination.geometry.y, destination.geometry.x}
Distance (meters): {distance}
Duration (seconds): {duration}
"""
)


OSRM results using profile=foot
------------
Origin: (-3.0547889500616674, -48.93935966253806),
Destination: (-3.051662, -48.937965)
Distance (meters): 273.7
Duration (seconds): 197.2



In [26]:
start_time = time.time()
distance_duration_para_by_foot = para_hexs.parallel_apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid,
        destination=filtered_schools.iloc[row["closest_school_id"]]["geometry"],
    ),
    result_type="expand",
    axis=1,
)
print("Elapsed time (s)", time.time() - start_time)

Waiting for server to be ready ...Waiting for server to be ready ...

Waiting for server to be ready ...
Waiting for server to be ready ...Waiting for server to be ready ...

Waiting for server to be ready ...Waiting for server to be ready ...

Waiting for server to be ready ...
Waiting for server to be ready ...Waiting for server to be ready ...

Elapsed time (s) 196.57561111450195


## Inspect results


In [27]:
para_hexs["distance_to_school_km_by_foot"] = (
    distance_duration_para_by_foot[0] / 1000
)  # meters to kilometers
para_hexs["duration_to_school_min_by_foot"] = (
    distance_duration_para_by_foot[1] / 60
)  # seconds to minutes

In [28]:
# Once we have finished with the OSRM server we stop it
up.routing.stop_osrm_server("norte", "south-america_brazil", PROFILE)

Server was stoped succesfully


## Create map for travel times by foot to nearest schools


In [29]:
para_hexs["duration_to_school_min_by_foot"].describe()

count    130363.000000
mean        259.309849
std         719.200650
min           0.000000
25%          15.275833
50%          67.283333
75%         209.785833
max       18042.888333
Name: duration_to_school_min_by_foot, dtype: float64

## Making map with bins of duration


First get default categories


In [30]:
custom_bins, custom_labels = up.utils.create_duration_labels(
    para_hexs["duration_to_school_min_by_foot"]
)
print(custom_bins)
print(custom_labels)

[0, 15, 30, 45, 60, 90, 120, 18043]
['0-15', '15-30', '30-45', '45-60', '60-90', '90-120', '>120']


Then convert from numerical to categorical


In [31]:
para_hexs["duration_to_school_min_by_foot_cat"] = pd.cut(
    para_hexs["duration_to_school_min_by_foot"], bins=custom_bins, labels=custom_labels
)

In [32]:
# Read hexagons and school data with geopandas
start_time = time.time()
microregions = gpd.read_parquet("outputs/para_micro_regions.parquets")
print(f"Read micro regions in {time.time() - start_time} seconds")

Read micro regions in 0.008533954620361328 seconds


In [33]:
# Import libraries for plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pydeck.data_utils import compute_view
from tqdm import tqdm
from unidecode import unidecode

In [34]:
# Calculate total number of students in each hexagon
para_hexs["QT_MAT_TOTAL"] = para_hexs[
    [
        "QT_MAT_BAS",
        "QT_MAT_INF",
        "QT_MAT_FUND",
        "QT_MAT_MED",
        "QT_MAT_PROF",
        "QT_MAT_EJA",
        "QT_MAT_ESP",
    ]
].sum(axis=1)

# Calculate total number of teachers in each hexagon
para_hexs["QT_DOC_TOTAL"] = para_hexs[
    [
        "QT_DOC_BAS",
        "QT_DOC_INF",
        "QT_DOC_FUND",
        "QT_DOC_MED",
        "QT_DOC_PROF",
        "QT_DOC_EJA",
        "QT_DOC_ESP",
    ]
].sum(axis=1)

# Calculate total number of schools in each hexagon
para_hexs["pob_school_age"] = para_hexs[
    [
        "pop_3_5_years_adj",
        "pop_6_14_years_adj",
        "pop_15_17_years_adj",
        "pop_18_years_adj",
    ]
].sum(axis=1)

In [35]:
# Calculate total number of students in schools
filtered_schools["QT_MAT_TOTAL"] = filtered_schools[
    [
        "QT_MAT_BAS",
        "QT_MAT_INF",
        "QT_MAT_FUND",
        "QT_MAT_MED",
        "QT_MAT_PROF",
        "QT_MAT_EJA",
        "QT_MAT_ESP",
    ]
].sum(axis=1)

# Calculate total number of teachers in schools
filtered_schools["QT_DOC_TOTAL"] = filtered_schools[
    [
        "QT_DOC_BAS",
        "QT_DOC_INF",
        "QT_DOC_FUND",
        "QT_DOC_MED",
        "QT_DOC_PROF",
        "QT_DOC_EJA",
        "QT_DOC_ESP",
    ]
].sum(axis=1)

In [36]:
# Use the total number of students to define the size of the map markers
SIZE_SCALE = 20
SIZE_MIN, SIZE_MAX = 5, 20
filtered_schools["marker_size"] = (
    filtered_schools["QT_MAT_TOTAL"] / filtered_schools["QT_MAT_TOTAL"].max()
) * SIZE_SCALE
filtered_schools["marker_size"] = np.clip(
    filtered_schools["marker_size"], SIZE_MIN, SIZE_MAX
)

In [37]:
size_brackets = {
    5: "0-2000",  # "0-1865",
    10: "2001-4000",  # "1865-3714",
    15: "4000-6000",  # "3772-5594",
    20: "6000-7500",  # "5873-7462",
}

In [38]:
output_folder = "outputs/para_microregions_maps/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# Create a visualization for each microregion
for ix, region in tqdm(microregions.iterrows(), total=len(microregions)):
    # Select the hexagons and schools that are within the microregion
    microregion_hexs = para_hexs[para_hexs.within(region.geometry)]
    microregions_schools = filtered_schools[filtered_schools.within(region.geometry)]

    # Compute the map center and zoom
    viewstate = compute_view(microregions_schools[["lon", "lat"]], view_proportion=0.8)

    # Calculate the population for each travel time category
    pop_by_duration = (
        microregion_hexs.groupby("duration_to_school_min_by_foot_cat")[
            "population_2020"
        ]
        .sum()
        .reset_index()
    )
    pop_by_duration["population_label"] = "Population"

    # Calculate the percentage of the total population for each travel time category
    pop_by_duration["prop_poulation_2020"] = (
        pop_by_duration["population_2020"]
        * 100
        / microregion_hexs["population_2020"].sum()
    ).round(2).astype(str) + " %"

    # Create a hexagon map colored by travel time to school categories
    hexs_fig = px.choropleth_mapbox(
        microregion_hexs,
        geojson=microregion_hexs.geometry,
        locations=microregion_hexs.index,
        opacity=0.6,
        color=microregion_hexs["duration_to_school_min_by_foot_cat"]
        .astype(str)
        .replace({"nan": np.nan}),
        color_discrete_sequence=px.colors.sequential.Viridis_r,
        category_orders={"color": custom_labels},
        hover_data=[
            "distance_to_school_km_by_foot",
            "duration_to_school_min_by_foot",
        ],
    )

    # Customize the legend and hover information
    hexs_fig.update_traces(
        legendgroup="Hexagons",
        legendgrouptitle_text="Travel Time (minutes)",
        hovertemplate="<b>Travel to closest school by foot</b><br>"
        + "Time: %{customdata[1]:.0f} minutes<br>"
        + "Distance: %{customdata[0]:.2f} kilometers<br>",
    )
    hexs_traces = hexs_fig.data

    # Create the schools points to be plotted on the map
    schools_trace = go.Scattermapbox(
        lat=microregions_schools["lat"],
        lon=microregions_schools["lon"],
        mode="markers",
        marker=dict(
            size=microregions_schools["marker_size"],
            color="green",
            opacity=0.5,
        ),
        showlegend=False,
        customdata=microregions_schools[
            [
                "name_school",
                "education_level",
                "admin_category",
                "government_level",
                "QT_MAT_TOTAL",
            ]
        ],
        hovertemplate="<b>%{customdata[0]}</b><br><br>"
        + "Education Level: %{customdata[1]}<br>"
        + "Administrative Category: %{customdata[2]}<br>"
        + "Government Level: %{customdata[3]}<br>"
        + "Number of Students: %{customdata[4]}"
        + "<extra></extra>",
    )

    # Create a bar chart with the population by travel time to school
    bar_trace = go.Bar(
        x=pop_by_duration["duration_to_school_min_by_foot_cat"],
        y=pop_by_duration["population_2020"],
        marker_color=px.colors.sequential.Viridis_r[: len(pop_by_duration)],
        text=pop_by_duration["prop_poulation_2020"],
        textposition="auto",
        xaxis="x2",
        yaxis="y2",
        hovertemplate="<b>%{y:,.0f} people, %{text}</b> of the total population,<br> have to <b>walk %{x} minutes</b> to reach the nearest school",
    )

    # Unify the map and the bar chart in a single figure
    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[0.3, 0.7],
        subplot_titles=[
            "Population by travel time to school",
            "Travel time to school by foot",
        ],
        specs=[[{"type": "bar"}, {"type": "mapbox"}]],
    )

    fig.add_trace(bar_trace, row=1, col=1)
    fig.update_traces(showlegend=False, row=1, col=1)

    fig.add_traces(hexs_traces, rows=1, cols=2)
    fig.update_traces(marker_line_width=0.0, row=1, col=2)
    fig.add_trace(schools_trace, row=1, col=2)
    # Create the legend with invisible traces
    fig.add_traces(
        [
            go.Scattermapbox(
                # Set a random point
                lat=[11.5],
                lon=[-77.5],
                mode="markers",
                marker=dict(
                    size=marker_size,
                    color="green",
                    opacity=1,
                ),
                legendgroup="Schools",
                legendgrouptitle_text="Schools (# students)",
                name=size_brackets[marker_size],
                showlegend=True,
            )
            for marker_size in range(5, 21, 5)
        ],
        rows=1,
        cols=2,
    )

    # Final customization of the figure
    fig.update_layout(
        margin=dict(l=20, r=20, t=30, b=20),
        mapbox=dict(
            style="open-street-map",
            zoom=viewstate.zoom,
            center=dict(lat=viewstate.latitude, lon=viewstate.longitude),
        ),
    )

    # Save the figure as an HTML file
    fig.write_html(output_folder + f"{unidecode(region['name_micro'].lower())}.html")

  0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 22/22 [01:45<00:00,  4.80s/it]


Then plot


In [ ]:
if False:
    map_figure = up.plotting.choropleth_map(
        para_hexs.reset_index(drop=True),
        "duration_to_school_min_by_foot_cat",
        zoom=8,
        opacity=0.6,
        title="Pará - Estimated travel times to school by foot",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        category_orders={"duration_to_school_min_by_foot_cat": custom_labels},
        labels={"duration_to_school_min_by_foot_cat": "Duration (minutes)"},
    )

    map_figure.update_layout(
        margin=dict(l=0, r=0, t=50, b=0),
    )
    map_figure.update_traces(marker_line_width=0)
    map_figure.show()

## Create map for travel times by car to nearest schools


In [ ]:
up.routing.start_osrm_server("norte", "south-america_brazil", "car")

In [ ]:
# # Stop all multiprocess child processes
# import multiprocessing
# active = multiprocessing.active_children()
# for child in active:
#     child.terminate()

In [ ]:
distance_duration_para_by_car = para_hexs.progress_apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid,
        destination=filtered_schools.iloc[row["closest_school_id"]]["geometry"],
    ),
    result_type="expand",
    axis=1,
)

In [ ]:
para_hexs["distance_to_school_km_by_car"] = (
    distance_duration_para_by_car[0] / 1000
)  # meters to kilometers
para_hexs["duration_to_school_min_by_car"] = (
    distance_duration_para_by_car[1] / 60
)  # seconds to minutes

In [ ]:
# Once we have finished with the OSRM server we stop it
up.routing.stop_osrm_server("norte", "south-america_brazil", "car")

## Create map for travel times by car to nearest schools


In [ ]:
para_hexs[
    [
        "distance_to_school_km_by_foot",
        "distance_to_school_km_by_car",
        "duration_to_school_min_by_foot",
        "duration_to_school_min_by_car",
    ]
].describe()

In [ ]:
if False:
    # Reset index is needed to avoid an error with plotly choropleth_map
    fig = up.plotting.choropleth_map(
        para_hexs.reset_index(drop=True),
        "duration_to_school_min_by_car",
        title="Para Estimated travel times to school by car",
        zoom=8,
        color_continuous_scale="Plasma_r",
        opacity=0.6,
        labels={"duration_to_school_min_by_car": "Duration (min)"},
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=50, b=0),
    )
    fig.update_traces(marker_line_width=0.0)
    fig.show()

## Making map with bins of duration


First get default categories


In [ ]:
custom_bins, custom_labels = up.utils.create_duration_labels(
    para_hexs["duration_to_school_min_by_car"]
)
print(custom_bins)
print(custom_labels)

Then convert from numerical to categorical


In [ ]:
para_hexs["duration_to_school_min_by_car_cat"] = pd.cut(
    para_hexs["duration_to_school_min_by_car"], bins=custom_bins, labels=custom_labels
)

In [ ]:
para_hexs["duration_to_school_min_by_foot_cat"].value_counts(normalize=True).round(
    4
) * 100

In [ ]:
para_hexs["duration_to_school_min_by_car_cat"].value_counts(normalize=True).round(
    4
) * 100

Then plot


In [ ]:
if False:
    map_figure = up.plotting.choropleth_map(
        para_hexs.reset_index(drop=True),
        "duration_to_school_min_by_car_cat",
        zoom=5,
        opacity=0.6,
        title="Pará - Estimated travel times to school by car",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        category_orders={"duration_to_school_min_by_car_cat": custom_labels},
        labels={"duration_to_school_min_by_car_cat": "Duration (minutes)"},
    )

    map_figure.update_layout(
        margin=dict(l=0, r=0, t=50, b=0),
    )
    map_figure.update_traces(marker_line_width=0.0)
    map_figure.show()

In [ ]:
para_hexs.to_csv("outputs/20240116_para_hexs_fund_public_travel_times.csv")
para_hexs.to_parquet("outputs/20240116_para_hexs_fund_public_travel_times.parquet")

## Calculate schools given a travel time/radius


In [ ]:
para_hexs = gpd.read_parquet(
    "outputs/20240116_para_hexs_fund_public_travel_times.parquet"
)

In [ ]:
para_hexs.head()

In [ ]:
profile = "car"
# Travel time speeds
speeds = {"foot": 5, "bike": 10, "car": 20}
speed = speeds[profile] * 1000 / 60  # km/h to m/min

travel_time_bins = [15, 30]  # in minutes

for bins in travel_time_bins:
    # Calculate the maximum distance
    max_distance = bins * speed  # in meters
    max_distance_degrees = max_distance / 111320  # 1 degree is ~111.32 km
    buffer_radius = max_distance_degrees / 2

    filtered_schools_subset_buffer = filtered_schools.buffer(buffer_radius)

    # Intersect buffers with hexagons to get the number of buffers per hexagon
    para_hexs[f"schools_within_{bins}min_travel_time_{profile}"] = (
        para_hexs.parallel_apply(
            lambda row: filtered_schools_subset_buffer.intersects(row.geometry).sum(),
            axis=1,
        )
    )

In [ ]:
para_hexs.head()

In [ ]:
para_hexs[
    ["schools_within_15min_travel_time_foot", "schools_within_30min_travel_time_foot"]
].describe()

In [ ]:
para_hexs.to_parquet(
    "outputs/20240116_para_hexs_fund_public_travel_times_counts.parquet"
)

In [ ]:
## Create school indicator aggregates for each hexagon

# Create the filters
# service_filter = (
#     br_schools["service_restriction"]
#     == "ESCOLA EM FUNCIONAMENTO E SEM RESTRIÇÃO DE ATENDIMENTO"
# )
# level_filter = br_schools["education_level"].str.contains("Ensino Fundamental")
# admin_filter = br_schools["admin_category"] == "Pública"

In [ ]:
filtered_schools.head()

In [ ]:
# Variables representing the number of students (for Fundamental level)
qt_students_fund = ["QT_MAT_FUND", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF"]

# Variables representing the number of teachers (for Fundamental level)
qt_teachers_fund = ["QT_DOC_FUND", "QT_DOC_FUND_AI", "QT_DOC_FUND_AF"]

# Variables representing the number of classes (for Fundamental level)
qt_classes_fun = ["QT_TUR_FUND", "QT_TUR_FUND_AI", "QT_TUR_FUND_AF"]

# Variables representing the 'Indicador de esforço docente' (for Fundamental level)
ied_fun = ["FUN_CAT_1", "FUN_CAT_2", "FUN_CAT_3", "FUN_CAT_4", "FUN_CAT_5", "FUN_CAT_6"]
# FUN_AI_CAT_1 FUN_AI_CAT_2 FUN_AI_CAT_3 FUN_AI_CAT_4 FUN_AI_CAT_5 FUN_AI_CAT_6
# FUN_AF_CAT_1 FUN_AF_CAT_2 FUN_AF_CAT_3 FUN_AF_CAT_4 FUN_AF_CAT_5 FUN_AF_CAT_6

# Variables representing the 'Média de Alunos por Turma' (for Fundamental level)
mat_fun = "FUN_CAT_0"
# 'FUN_AI_CAT_0', 'FUN_AF_CAT_0', 'MULT_ETA_CAT_0'
# FUN_01_CAT_0 FUN_02_CAT_0 FUN_03_CAT_0 FUN_04_CAT_0 FUN_05_CAT_0 FUN_06_CAT_0 FUN_07_CAT_0 FUN_08_CAT_0 FUN_09_CAT_0

In [ ]:
filtered_schools[
    ["QT_MAT_FUND", "QT_MAT_FUND_AI", "QT_MAT_FUND_AF"]
].isna().sum() / len(filtered_schools) * 100

In [ ]:
filtered_schools[
    ["QT_DOC_FUND", "QT_DOC_FUND_AI", "QT_DOC_FUND_AF"]
].isna().sum() / len(filtered_schools) * 100

In [ ]:
filtered_schools[
    ["QT_TUR_FUND", "QT_TUR_FUND_AI", "QT_TUR_FUND_AF"]
].isna().sum() / len(filtered_schools) * 100

In [ ]:
filtered_schools[
    ["FUN_CAT_1", "FUN_CAT_2", "FUN_CAT_3", "FUN_CAT_4", "FUN_CAT_5", "FUN_CAT_6"]
].isna().sum() / len(filtered_schools) * 100

In [ ]:
filtered_schools[
    ["FUN_CAT_0", "FUN_AI_CAT_0", "FUN_AF_CAT_0", "MULT_ETA_CAT_0"]
].isna().sum() / len(filtered_schools) * 100

In [ ]:
qt_students_fund
qt_teachers_fund
qt_classes_fun
postfix = ["_FUND", "_FUND_AI", "_FUND_AF"]

for i in range(len(qt_students_fund)):
    filtered_schools[f"students_per_professor{postfix[i]}"] = (
        filtered_schools[qt_students_fund[i]] / filtered_schools[qt_teachers_fund[i]]
    )
    filtered_schools[f"students_per_class{postfix[i]}"] = (
        filtered_schools[qt_students_fund[i]] / filtered_schools[qt_classes_fun[i]]
    )

In [ ]:
rename_indicators = {col: f"IED_NIV_{col.split('_')[-1]}_FUND" for col in ied_fun}
rename_indicators[mat_fun] = "MAT_FUND"

In [ ]:
# Rename columns
filtered_schools = filtered_schools.rename(columns=rename_indicators)

In [ ]:
# Get relevant columns
filtered_schools[
    [
        "students_per_professor_FUND",
        "students_per_class_FUND",
        "IED_NIV_1_FUND",
        "IED_NIV_2_FUND",
        "IED_NIV_3_FUND",
        "IED_NIV_4_FUND",
        "IED_NIV_5_FUND",
        "IED_NIV_6_FUND",
        "MAT_FUND",
    ]
].head()

In [ ]:
# Get relevant columns
filtered_schools[
    [
        "students_per_professor_FUND",
        "students_per_class_FUND",
        "IED_NIV_1_FUND",
        "IED_NIV_2_FUND",
        "IED_NIV_3_FUND",
        "IED_NIV_4_FUND",
        "IED_NIV_5_FUND",
        "IED_NIV_6_FUND",
        "MAT_FUND",
    ]
].describe()

In [ ]:
# import ax divider
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx

Description of teacher effort levels

| Level | Students | Shifts | Schools | Stages |
| ----- | -------- | ------ | ------- | ------ |
| **1** | <25      | 1      | 1       | 1      |
| **2** | 25-150   | 1      | 1       | 1      |
| **3** | 25-300   | 1-2    | 1       | 1      |
| **4** | 50-400   | 2      | 1-2     | 2      |
| **5** | >300     | 3      | 2-3     | 2-3    |
| **6** | >400     | 3      | 2-3     | 2-3    |


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharex=True, sharey=True)
fig.suptitle(
    "Percentage of teachers by levels of the teacher effort indicator - Educação Fundamental (Pública)"
)

# Create a single axes for the colorscale
cax = fig.add_axes([1, 0.1, 0.02, 0.8])

for i in range(6):
    ax = axes[i // 3, i % 3]
    filtered_schools.sort_values(f"IED_NIV_{i+1}_FUND").plot(
        f"IED_NIV_{i+1}_FUND", legend=True, ax=ax, cax=cax, markersize=1, cmap="magma"
    )
    ax.set_title(f"Nivel {i+1}")
    ax.set_axis_off()
    ctx.add_basemap(
        ax,
        crs=filtered_schools.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        attribution=False,
    )

plt.tight_layout()

In [ ]:
cols = ["MAT_FUND", "students_per_class_FUND"]
label = [
    "Média de Alunos por Turma (given)",
    "Number of students per class (calculated)",
]
fig, axes = plt.subplots(1, 2, figsize=(15, 7.5), sharex=True, sharey=True)
fig.suptitle("Number of Students per Class - Educação Fundamental (Pública)")


# Create a single axes for the colorscale
cax = fig.add_axes([1, 0.1, 0.02, 0.75])

for col, ax in zip(cols, axes):
    filtered_schools.sort_values(col).plot(
        col,
        legend=True,
        ax=ax,
        cax=cax,
        markersize=1,
        cmap="Reds",
        vmin=0,
        vmax=filtered_schools[col].max(),
    )
    cax_ticks = np.linspace(0, filtered_schools[col].max(), 5)
    cax.set_yticks(cax_ticks)
    cax.set_yticklabels([f"{int(tick):,}" for tick in cax_ticks])

    ax.set_title(label[cols.index(col)])
    ax.set_axis_off()
    ctx.add_basemap(
        ax,
        crs=filtered_schools.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        attribution=False,
    )

plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with two subplots
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the histogram of the first variable
ax.hist(
    filtered_schools["MAT_FUND"],
    bins=30,
    color="blue",
    alpha=0.5,
    label="Média de Alunos por Turma (given)",
)
ax.set_ylabel("Frequency")

# Plot the histogram of the second variable
ax.hist(
    filtered_schools["students_per_class_FUND"],
    bins=30,
    color="red",
    alpha=0.5,
    label="Number of students per class (calculated)",
)

# Add a title to the figure
fig.suptitle("Histogram Comparison")

# Add legend to the figure
ax.legend()

# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure
plt.show()

In [ ]:
col = "students_per_professor_FUND"
fig, ax = plt.subplots(figsize=(7.5, 7.5))

# Create a single axes for the colorscale
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)


ax = filtered_schools.sort_values(col).plot(
    col,
    legend=True,
    ax=ax,
    cax=cax,
    markersize=1,
    cmap="RdYlGn_r",
    norm=norm,
)

cax_ticks = np.concatenate(
    (np.linspace(0, 14, 4), np.linspace(14, filtered_schools[col].max(), 4))
)
cax.set_yticks(cax_ticks)
cax.set_yticklabels([f"{int(tick):,}" for tick in cax_ticks])

# Add a blue market for the mean value in the colorbar
mean_value = filtered_schools[col].mean()
cax.axhline(mean_value, color="blue", linestyle="--")
cax.text(
    1.2,
    mean_value + 1.5,
    f"{mean_value:.2f}",
    va="center",
    ha="left",
    color="blue",
    bbox=dict(facecolor="none", alpha=0),
)

ax.set_title("Number of Students per Teacher - Educação Fundamental (Pública)")
ax.set_axis_off()
ctx.add_basemap(
    ax,
    crs=filtered_schools.crs.to_string(),
    source=ctx.providers.CartoDB.Positron,
    attribution=False,
)

plt.tight_layout()

In [ ]:
# Save the clean filtered schools to a parquet file
final_selected_columns = [
    "code_school",
    "name_school",
    "education_level",
    "address",
    "government_level",
    "date_update",
    "students_per_professor_FUND",
    "students_per_class_FUND",
    "MAT_FUND",
    "IED_NIV_1_FUND",
    "IED_NIV_2_FUND",
    "IED_NIV_3_FUND",
    "IED_NIV_4_FUND",
    "IED_NIV_5_FUND",
    "IED_NIV_6_FUND",
    "QT_MAT_FUND",
    "QT_MAT_FUND_AI",
    "QT_MAT_FUND_AF",
    "QT_DOC_FUND",
    "QT_DOC_FUND_AI",
    "QT_DOC_FUND_AF",
    "QT_TUR_FUND",
    "QT_TUR_FUND_AI",
    "QT_TUR_FUND_AF",
    "geometry",
]
filtered_schools[final_selected_columns].to_parquet(
    "outputs/20240129_para_schools_final.parquet"
)

In [ ]:
para_hexs["territory_type"] = para_hexs["territory type"]
para_hexs["territory_type"].replace(
    {
        0: "Rural",
        1: "Urban",
    },
    inplace=True,
)

In [ ]:
access_var_labels = {
    "hex": "H3 Hexagon Index",
    "territory type": "Territory Type",
    "population_2020": "Population All Ages",
    "pop_6_14_years_adj": "Population Ages 6-14",
    "income_pc": "Avg Income Per Cápita (R$)",
    "ensino_fundamental": "# of schools - Ensino Fundamental",
    "duration_to_school_min_by_foot": "Travel time to the nearest school by foot",
    "duration_to_school_min_by_foot_cat": "Travel time to the nearest school by foot categories",
    "duration_to_school_min_by_car": "Travel time to the nearest school by car",
    "duration_to_school_min_by_car_cat": "Travel time to the nearest school by car categories",
    "schools_within_15min_travel_time_foot": "# of schools at <15 minutes by foot",
    "schools_within_30min_travel_time_foot": "# of schools at <30 minutes by foot",
    "schools_within_15min_travel_time_car": "# of schools at <15 minutes by car",
    "schools_within_30min_travel_time_car": "# of schools at <30 minutes by car",
}

In [ ]:
para_hexs_selected_cols = para_hexs[[*access_var_labels.keys(), "geometry"]]

In [ ]:
len(para_hexs_selected_cols.columns)

In [ ]:
para_hexs_selected_cols.select_dtypes(include="object").columns

In [ ]:
agg = {
    col: "sum"
    for col in para_hexs_selected_cols.select_dtypes(include="number").columns
}
agg["income_pc"] = "mean"
agg["duration_to_school_min_by_foot"] = "mean"
agg["duration_to_school_min_by_car"] = "mean"

In [ ]:
para_hexs_selected_cols_res7 = up.geom.resolution_downsampling(
    para_hexs_selected_cols, "hex", 4, agg=agg
)

In [ ]:
para_hexs_selected_cols_res7.head()

In [ ]:
# Create a plot for each variable
fig, axes = plt.subplots(
    nrows=agg.keys().__len__() // 3, ncols=3, figsize=(15, 15), sharex=True, sharey=True
)

for i, col in enumerate([*agg.keys()][2:]):
    ax = axes[i // 3, i % 3]
    if i % 2 == 0:
        para_hexs_selected_cols_res7.plot(
            col, legend=True, ax=ax, cmap="magma", edgecolor="none", linewidth=0.0
        )
    else:
        para_hexs_selected_cols_res7.plot(
            col,
            legend=True,
            ax=ax,
            cmap="magma",
        )
    ax.set_title(access_var_labels[col])
    ax.set_axis_off()
    ctx.add_basemap(
        ax,
        crs=para_hexs_selected_cols.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        attribution=False,
    )

In [ ]:
access_var_labels = {
    "hex": "H3 Hexagon Index",
    "territory type": "Territory Type",
    "population_2020": "Population All Ages",
    "pop_6_14_years_adj": "Population Ages 6-14",
    "income_pc": "Avg Income Per Cápita (R$)",
    "ensino_fundamental": "# of schools - Ensino Fundamental",
    "duration_to_school_min_by_foot": "Travel time to the nearest school by foot",
    "duration_to_school_min_by_foot_cat": "Travel time to the nearest school by foot categories",
    "duration_to_school_min_by_car": "Travel time to the nearest school by car",
    "duration_to_school_min_by_car_cat": "Travel time to the nearest school by car categories",
    "schools_within_15min_travel_time_foot": "# of schools at <15 minutes by foot",
    "schools_within_30min_travel_time_foot": "# of schools at <30 minutes by foot",
    "schools_within_15min_travel_time_car": "# of schools at <15 minutes by car",
    "schools_within_30min_travel_time_car": "# of schools at <30 minutes by car",
}

In [ ]:
# Aggregate school indicators by hexagon
schools_col = [
    "students_per_professor_FUND",
    "students_per_class_FUND",
    "MAT_FUND",
    "IED_NIV_1_FUND",
    "IED_NIV_2_FUND",
    "IED_NIV_3_FUND",
    "IED_NIV_4_FUND",
    "IED_NIV_5_FUND",
    "IED_NIV_6_FUND",
]
para_hexs_schools_cols = up.geom.merge_shape_hex(
    hexs=para_hexs_selected_cols,
    shape=filtered_schools,
    agg={col: "mean" for col in schools_col},
)

In [ ]:
para_hexs_schools_cols.geometry

In [ ]:
para_hexs_schools_cols_res4 = up.geom.resolution_downsampling(
    para_hexs_schools_cols, "hex", 4, agg={col: "mean" for col in schools_col}
)

In [ ]:
# Create a plot for each variable
fig, axes = plt.subplots(
    nrows=len(schools_col) // 3, ncols=3, figsize=(15, 15), sharex=True, sharey=True
)

for i, col in enumerate(schools_col):
    ax = axes[i // 3, i % 3]
    para_hexs_schools_cols_res4.plot(
        col,
        legend=True,
        ax=ax,
        cmap="magma",
    )
    ax.set_title(col)
    ax.set_axis_off()
    ctx.add_basemap(
        ax,
        crs=para_hexs_selected_cols.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        attribution=False,
    )

In [ ]:
capacity_var_labels = {
    "students_per_professor_FUND": "Average # Students per Teacher",
    "students_per_class_FUND": "Average # Students per Class (calculated)",
    "MAT_FUND": "Average # Students per Class (given)",
    "IED_NIV_1_FUND": "Perc. Teachers with effort indicator - Level 1",
    "IED_NIV_2_FUND": "Perc. Teachers with effort indicator - Level 2",
    "IED_NIV_3_FUND": "Perc. Teachers with effort indicator - Level 3",
    "IED_NIV_4_FUND": "Perc. Teachers with effort indicator - Level 4",
    "IED_NIV_5_FUND": "Perc. Teachers with effort indicator - Level 5",
    "IED_NIV_6_FUND": "Perc. Teachers with effort indicator - Level 6",
}

In [ ]:
# Save dataset as parquet file
para_hexs_schools_cols.to_parquet(
    "outputs/20240129_para_hexs_with_accessibility_capacity_vars.parquet"
)

In [ ]:
para_hexs_schools_cols.geometry